In [83]:
# Importing the required libraries

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import datetime
from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns             # statistical data visualization

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import statsmodels.api as sn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [84]:
# Converting process excel file to csv

read_file = pd.read_excel ('Data_for_pred.xlsx', sheet_name = 'Compiled_Sheet')
read_file.to_csv ('data_for_pred.csv', index = None, header=True)

In [453]:
# Importing the dataframes

df = pd.read_csv('data_for_pred.csv')

In [454]:
df.columns

Index(['Date', 'Freight Rate', 'LME_Aluminium', 'LME_Al_Stock', 'LME_Copper',
       'LME_Cu_Stock', 'LME_Zinc', 'LME_Zn', 'EUR_USD', 'USD_INR',
       'USD_INR_Volume', 'Coal_price', 'Crude_oil_WTI_price',
       'Crude_oil_WTI_Vol', 'Natural_gas_price', 'Natural_gas_Volume',
       'Crude_oil_Brent_price', 'Crude_oil_Brent_Vol'],
      dtype='object')

In [455]:
last_10_days = df.tail(10)
last_10_days.reset_index(inplace = True)
last_10_days = last_10_days.drop(columns=['index'])

In [456]:
last_10_days

,Date,Freight Rate,LME_Aluminium,LME_Al_Stock,LME_Copper,LME_Cu_Stock,LME_Zinc,LME_Zn,EUR_USD,USD_INR,USD_INR_Volume,Coal_price,Crude_oil_WTI_price,Crude_oil_WTI_Vol,Natural_gas_price,Natural_gas_Volume,Crude_oil_Brent_price,Crude_oil_Brent_Vol
0,2023-07-05,1050,2093.0,539700,8251.0,67200,2357.0,76475,1.0879,82.34,2090000.0,0.742045,71.79,456810.0,2.657,124910.0,76.65,293740.0
1,2023-07-06,1050,2103.5,537875,8280.0,65175,2365.0,74775,1.0896,82.51,2950000.0,0.741728,71.80,395380.0,2.609,109300.0,76.52,124500.0
2,2023-07-07,1050,2086.0,535075,8299.0,62975,2337.5,73975,1.0888,82.79,2290000.0,0.730161,73.86,362630.0,2.582,103590.0,78.47,331530.0
3,2023-07-10,1050,2096.0,534075,8286.0,59425,2341.5,73500,1.0955,82.65,1590000.0,0.747126,72.99,301010.0,2.669,124500.0,77.69,288400.0
4,2023-07-11,1050,2129.0,531725,8371.0,57075,2361.0,72925,1.0983,82.46,2160000.0,0.757943,74.83,326880.0,2.731,130580.0,79.40,333840.0
5,2023-07-12,1050,2147.5,528100,8367.0,54450,2360.0,72150,1.1016,82.33,1640000.0,0.763999,75.75,352890.0,2.632,149630.0,80.11,362330.0
6,2023-07-13,1100,2236.0,532100,8585.0,54225,2462.0,71750,1.1174,82.12,2490000.0,0.777521,76.89,379730.0,2.545,148020.0,81.36,416120.0
7,2023-07-14,1100,2225.0,528475,8651.0,57175,2413.0,70775,1.1224,82.21,2180000.0,0.782143,75.42,299580.0,2.539,134590.0,79.87,311230.0
8,2023-07-17,1100,2195.0,520800,8460.5,59175,2354.0,69725,1.1235,82.09,2070000.0,0.771714,74.15,217420.0,2.512,97500.0,78.50,341160.0
9,2023-07-18,1100,2169.5,517400,8410.0,59200,2348.0,80375,1.1253,82.07,1740000.0,0.765810,75.75,125090.0,2.629,134870.0,79.63,366470.0


In [457]:
# graph_df = df[1220:]

In [458]:
# for col in graph_df.columns[1:]:
#     sns.jointplot(x = 'Date', y = col, data = graph_df)

In [459]:
# for col in df.columns:
#     print ("d['" + col + "']", end = ", ")

In [460]:
# date for prediction
for i in range(10):
    last_rec_date = last_10_days['Date'][len(last_10_days)-1]
#     print (last_rec_date)
    date_split = last_rec_date.split('-')
    date_split[2] = str(int(date_split[2]) + 1)
    if (len(date_split[2]) == 1):
        date_split[2] = '0' + date_split[2]
    next_date = "-".join(date_split)
    print (next_date)
    
    
    n = len(df)
    d = {}
    exp_func = [0.33,0.09,-0.42,0.42,0.14,0.28,-0.51,0.35,-0.13,0.45]
#     exp_func = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
    
    for col in df.columns[2:]:
        num = 0
        for j,factor in zip(range(10), exp_func):
            num += float(last_10_days[col][i+j])*factor
        d[col] = num
#     print (d)
    
    df_test = pd.DataFrame([[next_date, df['Freight Rate'][n-1], d['LME_Aluminium'], d['LME_Al_Stock'], d['LME_Copper'], d['LME_Cu_Stock'], d['LME_Zinc'], d['LME_Zn'], d['EUR_USD'], d['USD_INR'], d['USD_INR_Volume'], d['Coal_price'], d['Crude_oil_WTI_price'], d['Crude_oil_WTI_Vol'], d['Natural_gas_price'], d['Natural_gas_Volume'], d['Crude_oil_Brent_price'], d['Crude_oil_Brent_Vol']]],columns=df.columns)
    last_10_days = pd.concat([last_10_days, df_test], ignore_index = True)

2023-07-19
2023-07-20
2023-07-21
2023-07-22
2023-07-23
2023-07-24
2023-07-25
2023-07-26
2023-07-27
2023-07-28


In [461]:
last_10_days

,Date,Freight Rate,LME_Aluminium,LME_Al_Stock,LME_Copper,LME_Cu_Stock,LME_Zinc,LME_Zn,EUR_USD,USD_INR,USD_INR_Volume,Coal_price,Crude_oil_WTI_price,Crude_oil_WTI_Vol,Natural_gas_price,Natural_gas_Volume,Crude_oil_Brent_price,Crude_oil_Brent_Vol
0,2023-07-05,1050,2093.000000,539700.000000,8251.000000,67200.000000,2357.000000,76475.000000,1.087900,82.340000,2.090000e+06,0.742045,71.790000,456810.000000,2.657000,124910.000000,76.650000,293740.000000
1,2023-07-06,1050,2103.500000,537875.000000,8280.000000,65175.000000,2365.000000,74775.000000,1.089600,82.510000,2.950000e+06,0.741728,71.800000,395380.000000,2.609000,109300.000000,76.520000,124500.000000
2,2023-07-07,1050,2086.000000,535075.000000,8299.000000,62975.000000,2337.500000,73975.000000,1.088800,82.790000,2.290000e+06,0.730161,73.860000,362630.000000,2.582000,103590.000000,78.470000,331530.000000
3,2023-07-10,1050,2096.000000,534075.000000,8286.000000,59425.000000,2341.500000,73500.000000,1.095500,82.650000,1.590000e+06,0.747126,72.990000,301010.000000,2.669000,124500.000000,77.690000,288400.000000
4,2023-07-11,1050,2129.000000,531725.000000,8371.000000,57075.000000,2361.000000,72925.000000,1.098300,82.460000,2.160000e+06,0.757943,74.830000,326880.000000,2.731000,130580.000000,79.400000,333840.000000
5,2023-07-12,1050,2147.500000,528100.000000,8367.000000,54450.000000,2360.000000,72150.000000,1.101600,82.330000,1.640000e+06,0.763999,75.750000,352890.000000,2.632000,149630.000000,80.110000,362330.000000
6,2023-07-13,1100,2236.000000,532100.000000,8585.000000,54225.000000,2462.000000,71750.000000,1.117400,82.120000,2.490000e+06,0.777521,76.890000,379730.000000,2.545000,148020.000000,81.360000,416120.000000
7,2023-07-14,1100,2225.000000,528475.000000,8651.000000,57175.000000,2413.000000,70775.000000,1.122400,82.210000,2.180000e+06,0.782143,75.420000,299580.000000,2.539000,134590.000000,79.870000,311230.000000
8,2023-07-17,1100,2195.000000,520800.000000,8460.500000,59175.000000,2354.000000,69725.000000,1.123500,82.090000,2.070000e+06,0.771714,74.150000,217420.000000,2.512000,97500.000000,78.500000,341160.000000
9,2023-07-18,1100,2169.500000,517400.000000,8410.000000,59200.000000,2348.000000,80375.000000,1.125300,82.070000,1.740000e+06,0.765810,75.750000,125090.000000,2.629000,134870.000000,79.630000,366470.000000


In [462]:
# n = len(df)
# df_test = pd.DataFrame([[next_date, df['Freight Rate'][n-1], df['LME_Aluminium'][n-1], df['LME_Al_Stock'][n-1], df['LME_Copper'][n-1], df['LME_Cu_Stock'][n-1], df['LME_Zinc'][n-1], df['LME_Zn'][n-1], df['EUR_USD'][n-1], df['USD_INR'][n-1], df['USD_INR_Volume'][n-1], df['Coal_price'][n-1], df['Crude_oil_WTI_price'][n-1], df['Crude_oil_WTI_Vol'][n-1], df['Natural_gas_price'][n-1], df['Natural_gas_Volume'][n-1], df['Crude_oil_Brent_price'][n-1], df['Crude_oil_Brent_Vol'][n-1]]],columns=df.columns)
# df = pd.concat([df, df_test], ignore_index = True)

In [463]:
df = pd.concat([df, last_10_days[10:]], ignore_index = True)

In [464]:
df

,Date,Freight Rate,LME_Aluminium,LME_Al_Stock,LME_Copper,LME_Cu_Stock,LME_Zinc,LME_Zn,EUR_USD,USD_INR,USD_INR_Volume,Coal_price,Crude_oil_WTI_price,Crude_oil_WTI_Vol,Natural_gas_price,Natural_gas_Volume,Crude_oil_Brent_price,Crude_oil_Brent_Vol
0,2018-01-02,1500,2256.000000,1.101375e+06,7181.000000,200650.000000,3377.000000,180975.000000,1.206300,63.660000,1.340000e+06,0.880459,60.370000,510310.000000,3.056000,320180.000000,66.570000,211740.000000
1,2018-01-03,1500,2241.000000,1.101800e+06,7115.500000,200225.000000,3350.500000,180825.000000,1.202300,63.700000,1.160000e+06,0.880691,61.630000,673860.000000,3.008000,269890.000000,67.840000,260860.000000
2,2018-01-04,1500,2230.000000,1.101325e+06,7202.500000,200150.000000,3377.000000,180575.000000,1.207100,63.560000,1.050000e+06,0.888924,62.010000,654360.000000,2.880000,319710.000000,68.070000,246150.000000
3,2018-01-05,1500,2205.500000,1.099475e+06,7097.000000,203850.000000,3396.000000,180325.000000,1.204700,63.520000,8.747300e+05,0.887122,61.440000,563040.000000,2.795000,230950.000000,67.620000,183450.000000
4,2018-01-08,1500,2177.500000,1.095875e+06,7084.500000,203850.000000,3391.500000,180325.000000,1.197100,63.600000,9.917300e+05,0.889937,61.730000,616470.000000,2.835000,228240.000000,67.780000,210350.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,2023-07-24,1100,2153.042313,5.224522e+05,8375.417033,57501.155895,2341.668413,74396.208575,1.113247,82.257410,1.640276e+06,0.762071,75.194685,230875.979013,2.598759,128151.251218,79.391620,367284.893437
1403,2023-07-25,1100,2171.140389,5.272352e+05,8432.737016,57565.838303,2385.003050,76243.257121,1.114240,82.184266,1.891014e+06,0.768192,75.223106,263404.902069,2.652187,143789.650974,79.541392,349029.249794
1404,2023-07-26,1100,2156.165015,5.290543e+05,8429.824188,58742.432355,2364.664445,73388.146455,1.107186,82.272489,2.020626e+06,0.766920,74.358361,313730.643134,2.648507,134894.508374,78.816668,284831.145432
1405,2023-07-27,1100,2166.801754,5.272423e+05,8395.065775,58335.446341,2372.239306,72384.558080,1.108968,82.225484,2.193002e+06,0.761790,74.885202,319655.384059,2.565151,122428.391902,79.272599,343966.731303


In [465]:
for col in df.columns[2:]:
    df[col + '_shift-' + str(1) ] = df[col].shift(1) 

In [466]:
df = df.drop(columns = ['LME_Aluminium', 'LME_Al_Stock', 'LME_Copper',
       'LME_Cu_Stock', 'LME_Zinc', 'LME_Zn', 'EUR_USD', 'USD_INR',
       'USD_INR_Volume', 'Coal_price', 'Crude_oil_WTI_price',
       'Crude_oil_WTI_Vol', 'Natural_gas_price', 'Natural_gas_Volume',
       'Crude_oil_Brent_price', 'Crude_oil_Brent_Vol'])
df

,Date,Freight Rate,LME_Aluminium_shift-1,LME_Al_Stock_shift-1,LME_Copper_shift-1,LME_Cu_Stock_shift-1,LME_Zinc_shift-1,LME_Zn_shift-1,EUR_USD_shift-1,USD_INR_shift-1,USD_INR_Volume_shift-1,Coal_price_shift-1,Crude_oil_WTI_price_shift-1,Crude_oil_WTI_Vol_shift-1,Natural_gas_price_shift-1,Natural_gas_Volume_shift-1,Crude_oil_Brent_price_shift-1,Crude_oil_Brent_Vol_shift-1
0,2018-01-02,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03,1500,2256.000000,1.101375e+06,7181.000000,200650.000000,3377.000000,180975.000000,1.206300,63.660000,1.340000e+06,0.880459,60.370000,510310.000000,3.056000,320180.000000,66.570000,211740.000000
2,2018-01-04,1500,2241.000000,1.101800e+06,7115.500000,200225.000000,3350.500000,180825.000000,1.202300,63.700000,1.160000e+06,0.880691,61.630000,673860.000000,3.008000,269890.000000,67.840000,260860.000000
3,2018-01-05,1500,2230.000000,1.101325e+06,7202.500000,200150.000000,3377.000000,180575.000000,1.207100,63.560000,1.050000e+06,0.888924,62.010000,654360.000000,2.880000,319710.000000,68.070000,246150.000000
4,2018-01-08,1500,2205.500000,1.099475e+06,7097.000000,203850.000000,3396.000000,180325.000000,1.204700,63.520000,8.747300e+05,0.887122,61.440000,563040.000000,2.795000,230950.000000,67.620000,183450.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,2023-07-24,1100,2168.917066,5.283119e+05,8467.572425,58687.596681,2369.604736,70808.824723,1.110406,82.346454,2.329292e+06,0.764495,74.500711,292471.880643,2.595555,112257.153482,79.009936,311999.028046
1403,2023-07-25,1100,2153.042313,5.224522e+05,8375.417033,57501.155895,2341.668413,74396.208575,1.113247,82.257410,1.640276e+06,0.762071,75.194685,230875.979013,2.598759,128151.251218,79.391620,367284.893437
1404,2023-07-26,1100,2171.140389,5.272352e+05,8432.737016,57565.838303,2385.003050,76243.257121,1.114240,82.184266,1.891014e+06,0.768192,75.223106,263404.902069,2.652187,143789.650974,79.541392,349029.249794
1405,2023-07-27,1100,2156.165015,5.290543e+05,8429.824188,58742.432355,2364.664445,73388.146455,1.107186,82.272489,2.020626e+06,0.766920,74.358361,313730.643134,2.648507,134894.508374,78.816668,284831.145432


In [467]:
df.columns = ['Date', 'Freight Rate', 'LME_Aluminium', 'LME_Al_Stock', 'LME_Copper',
       'LME_Cu_Stock', 'LME_Zinc', 'LME_Zn', 'EUR_USD', 'USD_INR',
       'USD_INR_Volume', 'Coal_price', 'Crude_oil_WTI_price',
       'Crude_oil_WTI_Vol', 'Natural_gas_price', 'Natural_gas_Volume',
       'Crude_oil_Brent_price', 'Crude_oil_Brent_Vol']

In [468]:
df = df[1:]

In [469]:
df.reset_index(inplace = True)
df = df.drop(columns=['index'])

In [470]:
df

,Date,Freight Rate,LME_Aluminium,LME_Al_Stock,LME_Copper,LME_Cu_Stock,LME_Zinc,LME_Zn,EUR_USD,USD_INR,USD_INR_Volume,Coal_price,Crude_oil_WTI_price,Crude_oil_WTI_Vol,Natural_gas_price,Natural_gas_Volume,Crude_oil_Brent_price,Crude_oil_Brent_Vol
0,2018-01-03,1500,2256.000000,1.101375e+06,7181.000000,200650.000000,3377.000000,180975.000000,1.206300,63.660000,1.340000e+06,0.880459,60.370000,510310.000000,3.056000,320180.000000,66.570000,211740.000000
1,2018-01-04,1500,2241.000000,1.101800e+06,7115.500000,200225.000000,3350.500000,180825.000000,1.202300,63.700000,1.160000e+06,0.880691,61.630000,673860.000000,3.008000,269890.000000,67.840000,260860.000000
2,2018-01-05,1500,2230.000000,1.101325e+06,7202.500000,200150.000000,3377.000000,180575.000000,1.207100,63.560000,1.050000e+06,0.888924,62.010000,654360.000000,2.880000,319710.000000,68.070000,246150.000000
3,2018-01-08,1500,2205.500000,1.099475e+06,7097.000000,203850.000000,3396.000000,180325.000000,1.204700,63.520000,8.747300e+05,0.887122,61.440000,563040.000000,2.795000,230950.000000,67.620000,183450.000000
4,2018-01-09,1500,2177.500000,1.095875e+06,7084.500000,203850.000000,3391.500000,180325.000000,1.197100,63.600000,9.917300e+05,0.889937,61.730000,616470.000000,2.835000,228240.000000,67.780000,210350.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,2023-07-24,1100,2168.917066,5.283119e+05,8467.572425,58687.596681,2369.604736,70808.824723,1.110406,82.346454,2.329292e+06,0.764495,74.500711,292471.880643,2.595555,112257.153482,79.009936,311999.028046
1402,2023-07-25,1100,2153.042313,5.224522e+05,8375.417033,57501.155895,2341.668413,74396.208575,1.113247,82.257410,1.640276e+06,0.762071,75.194685,230875.979013,2.598759,128151.251218,79.391620,367284.893437
1403,2023-07-26,1100,2171.140389,5.272352e+05,8432.737016,57565.838303,2385.003050,76243.257121,1.114240,82.184266,1.891014e+06,0.768192,75.223106,263404.902069,2.652187,143789.650974,79.541392,349029.249794
1404,2023-07-27,1100,2156.165015,5.290543e+05,8429.824188,58742.432355,2364.664445,73388.146455,1.107186,82.272489,2.020626e+06,0.766920,74.358361,313730.643134,2.648507,134894.508374,78.816668,284831.145432


In [471]:
df

,Date,Freight Rate,LME_Aluminium,LME_Al_Stock,LME_Copper,LME_Cu_Stock,LME_Zinc,LME_Zn,EUR_USD,USD_INR,USD_INR_Volume,Coal_price,Crude_oil_WTI_price,Crude_oil_WTI_Vol,Natural_gas_price,Natural_gas_Volume,Crude_oil_Brent_price,Crude_oil_Brent_Vol
0,2018-01-03,1500,2256.000000,1.101375e+06,7181.000000,200650.000000,3377.000000,180975.000000,1.206300,63.660000,1.340000e+06,0.880459,60.370000,510310.000000,3.056000,320180.000000,66.570000,211740.000000
1,2018-01-04,1500,2241.000000,1.101800e+06,7115.500000,200225.000000,3350.500000,180825.000000,1.202300,63.700000,1.160000e+06,0.880691,61.630000,673860.000000,3.008000,269890.000000,67.840000,260860.000000
2,2018-01-05,1500,2230.000000,1.101325e+06,7202.500000,200150.000000,3377.000000,180575.000000,1.207100,63.560000,1.050000e+06,0.888924,62.010000,654360.000000,2.880000,319710.000000,68.070000,246150.000000
3,2018-01-08,1500,2205.500000,1.099475e+06,7097.000000,203850.000000,3396.000000,180325.000000,1.204700,63.520000,8.747300e+05,0.887122,61.440000,563040.000000,2.795000,230950.000000,67.620000,183450.000000
4,2018-01-09,1500,2177.500000,1.095875e+06,7084.500000,203850.000000,3391.500000,180325.000000,1.197100,63.600000,9.917300e+05,0.889937,61.730000,616470.000000,2.835000,228240.000000,67.780000,210350.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,2023-07-24,1100,2168.917066,5.283119e+05,8467.572425,58687.596681,2369.604736,70808.824723,1.110406,82.346454,2.329292e+06,0.764495,74.500711,292471.880643,2.595555,112257.153482,79.009936,311999.028046
1402,2023-07-25,1100,2153.042313,5.224522e+05,8375.417033,57501.155895,2341.668413,74396.208575,1.113247,82.257410,1.640276e+06,0.762071,75.194685,230875.979013,2.598759,128151.251218,79.391620,367284.893437
1403,2023-07-26,1100,2171.140389,5.272352e+05,8432.737016,57565.838303,2385.003050,76243.257121,1.114240,82.184266,1.891014e+06,0.768192,75.223106,263404.902069,2.652187,143789.650974,79.541392,349029.249794
1404,2023-07-27,1100,2156.165015,5.290543e+05,8429.824188,58742.432355,2364.664445,73388.146455,1.107186,82.272489,2.020626e+06,0.766920,74.358361,313730.643134,2.648507,134894.508374,78.816668,284831.145432


In [472]:
# for col in df.columns[1:]:
#     sns.jointplot(x = 'Date', y = col, data = df)

In [473]:
def duplicate_var(df, var):
    df['temp'] = df[var]
    for i in range(1, len(df)):
        if df[var][i] == df[var][i-1]:
            df['temp'][i] = np.nan
    df[var] = df['temp']
    del df['temp']
    return df

def organize(df):
        
    # Took this dataset as y_variable
    del temp_df['Freight Rate']
      
    return pd.DataFrame(df)

def scaling_and_normalizing(df):
    min_max_scaler = preprocessing.MinMaxScaler()
    scaled_array = min_max_scaler.fit_transform(df)
    df_normalized = pd.DataFrame(scaled_array)
    return df_normalized

def concat(df1, df2):
    df = pd.concat([df1, df2], axis = 1, join = 'inner')
    return df

df = duplicate_var(df, 'Freight Rate')

C:\Users\218934\AppData\Local\Temp/ipykernel_22456/2999862869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp'][i] = np.nan
C:\Users\218934\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [474]:
df['Freight Rate'].interpolate(method= 'linear', direction = 'forward', inplace=True)

In [475]:
temp_df = df.copy()

moving_avg = [3, 5, 7, 10, 15, 20]

for col in temp_df.columns[2:]:
    for i in moving_avg:
        temp_df[col + '_moving_avg-' + str(i) ] = temp_df[col].rolling(i).mean()

In [476]:
del temp_df['Date']
temp_df = temp_df[20:]

cols_list = list(temp_df.columns)
cols_list.remove('Freight Rate')

rate_df = temp_df['Freight Rate']

temp_df = organize(temp_df)

# Scaling and normalizing the dataframe
temp_df = scaling_and_normalizing(temp_df)

In [477]:
# Initiating the column name to the dataframe
temp_df.columns = cols_list

In [478]:
temp_df = concat(rate_df, temp_df)

temp_df = temp_df.drop(columns = ['LME_Aluminium', 'LME_Al_Stock', 'LME_Copper',
       'LME_Cu_Stock', 'LME_Zinc', 'LME_Zn', 'EUR_USD', 'USD_INR',
       'USD_INR_Volume', 'Coal_price', 'Crude_oil_WTI_price',
       'Crude_oil_WTI_Vol', 'Natural_gas_price', 'Natural_gas_Volume',
       'Crude_oil_Brent_price', 'Crude_oil_Brent_Vol'])

In [479]:
temp_df

,Freight Rate,LME_Aluminium_moving_avg-3,LME_Aluminium_moving_avg-5,LME_Aluminium_moving_avg-7,LME_Aluminium_moving_avg-10,LME_Aluminium_moving_avg-15,LME_Aluminium_moving_avg-20,LME_Al_Stock_moving_avg-3,LME_Al_Stock_moving_avg-5,LME_Al_Stock_moving_avg-7,...,Crude_oil_Brent_price_moving_avg-7,Crude_oil_Brent_price_moving_avg-10,Crude_oil_Brent_price_moving_avg-15,Crude_oil_Brent_price_moving_avg-20,Crude_oil_Brent_Vol_moving_avg-3,Crude_oil_Brent_Vol_moving_avg-5,Crude_oil_Brent_Vol_moving_avg-7,Crude_oil_Brent_Vol_moving_avg-10,Crude_oil_Brent_Vol_moving_avg-15,Crude_oil_Brent_Vol_moving_avg-20
20,1590.000000,0.312496,0.328444,0.343355,0.346801,0.349504,0.352407,0.622900,0.623297,0.623372,...,0.450971,0.439247,0.425974,0.433346,0.207555,0.254631,0.238961,0.280108,0.497856,0.603398
21,1600.000000,0.305404,0.325702,0.336486,0.347744,0.348774,0.350830,0.621637,0.624484,0.625857,...,0.451126,0.440681,0.426162,0.431587,0.113644,0.169667,0.205380,0.188589,0.368545,0.605089
22,1593.333333,0.299139,0.321262,0.330351,0.346778,0.347979,0.349038,0.620715,0.625428,0.627022,...,0.449117,0.440176,0.425481,0.428455,0.195887,0.203877,0.223087,0.202261,0.367265,0.606073
23,1586.666667,0.295215,0.315517,0.327412,0.344686,0.348043,0.347030,0.621637,0.624496,0.627754,...,0.447631,0.439701,0.426579,0.426190,0.282100,0.229023,0.230097,0.209885,0.335409,0.596174
24,1580.000000,0.291979,0.310860,0.323642,0.338661,0.348218,0.345429,0.621262,0.623990,0.628468,...,0.446429,0.439570,0.428629,0.425071,0.407977,0.257318,0.232853,0.248412,0.314204,0.578059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,1054.166667,0.299754,0.307647,0.319111,0.334733,0.333638,0.328154,0.153329,0.152792,0.151415,...,0.574713,0.575841,0.568828,0.561392,0.457565,0.451395,0.512008,0.566546,0.581226,0.526739
1382,1058.333333,0.301252,0.311143,0.316430,0.334987,0.335546,0.328430,0.151608,0.152236,0.151556,...,0.575974,0.575116,0.570733,0.564277,0.471105,0.494873,0.520938,0.567956,0.598825,0.593739
1383,1062.500000,0.301769,0.315181,0.316535,0.332004,0.337695,0.329998,0.149820,0.151872,0.152397,...,0.575849,0.573279,0.572833,0.567076,0.494116,0.519674,0.514977,0.548870,0.652555,0.660819
1384,1066.666667,0.300013,0.316400,0.319301,0.328839,0.339924,0.331424,0.149967,0.151620,0.152563,...,0.577726,0.572215,0.573074,0.569183,0.479481,0.497383,0.525008,0.541360,0.641380,0.726434


In [480]:
X_multi = temp_df.drop("Freight Rate", axis=1)
y_multi = temp_df['Freight Rate']
X_multi_cons = sn.add_constant(X_multi)
# X_multi_cons.head()

C:\Users\218934\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [481]:
lm_multi = sn.OLS(y_multi, X_multi_cons).fit()
lm_multi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Freight Rate   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     172.2
Date:                Sat, 22 Jul 2023   Prob (F-statistic):               0.00
Time:                        17:45:17   Log-Likelihood:                -12053.
No. Observations:                1366   AIC:                         2.430e+04
Df Residuals:                    1269   BIC:                         2.481e+04
Df Model:                          96                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                1.326e+04   1538.462      8.617      0.000    1.02e+04    1.63e+04
LME_Aluminium_moving_avg-3            1.68e+04   9518.976      1.765      0.078   -1878.000    3.55e+04
LME_Aluminium_moving_avg-5          -5759.2122   1.98e+04     -0.291      0.771   -4.46e+04    3.31e+04
LME_Aluminium_moving_avg-7          -1288.9384   2.22e+04     -0.058      0.954   -4.47e+04    4.22e+04
LME_Aluminium_moving_avg-10          3234.8026   1.87e+04      0.173      0.863   -3.35e+04       4e+04
LME_Aluminium_moving_avg-15          2.765e+04   1.65e+04      1.673      0.095   -4770.336    6.01e+04
LME_Aluminium_moving_avg-20         -3.242e+04   9534.481     -3.400      0.001   -5.11e+04   -1.37e+04
LME_Al_Stock_moving_avg-3            6236.1953   9853.775      0.633      0.527   -1.31e+04    2.56e+04
LME_Al_Stock_moving_avg-5           -2888.1192   2.27e+04     -0.127      0.899   -4.73e+04    4.16e+04
LME_Al_Stock_moving_avg-7           -6642.5180   2.52e+04     -0.264      0.792    -5.6e+04    4.27e+04
LME_Al_Stock_moving_avg-10           1.482e+04   1.98e+04      0.750      0.454    -2.4e+04    5.36e+04
LME_Al_Stock_moving_avg-15          -2.899e+04   1.57e+04     -1.844      0.065   -5.98e+04    1858.042
LME_Al_Stock_moving_avg-20           2.176e+04   7730.015      2.815      0.005    6597.650    3.69e+04
LME_Copper_moving_avg-3             -3670.2080   9705.986     -0.378      0.705   -2.27e+04    1.54e+04
LME_Copper_moving_avg-5              8697.9354   2.06e+04      0.423      0.673   -3.17e+04    4.91e+04
LME_Copper_moving_avg-7             -8208.2484   2.34e+04     -0.350      0.726   -5.42e+04    3.77e+04
LME_Copper_moving_avg-10             7782.2276      2e+04      0.389      0.697   -3.15e+04     4.7e+04
LME_Copper_moving_avg-15            -6.305e+04   1.86e+04     -3.394      0.001   -9.95e+04   -2.66e+04
LME_Copper_moving_avg-20             8.078e+04   1.06e+04      7.620      0.000       6e+04    1.02e+05
LME_Cu_Stock_moving_avg-3           -1890.0322   6106.425     -0.310      0.757   -1.39e+04    1.01e+04
LME_Cu_Stock_moving_avg-5            -533.4074   1.39e+04     -0.038      0.969   -2.77e+04    2.67e+04
LME_Cu_Stock_moving_avg-7            4337.4138   1.55e+04      0.280      0.779    -2.6e+04    3.47e+04
LME_Cu_Stock_moving_avg-10          -7936.2659   1.23e+04     -0.647      0.518    -3.2e+04    1.61e+04
LME_Cu_Stock_moving_avg-15           1912.4287   9917.677      0.193      0.847   -1.75e+04    2.14e+04
LME_Cu_Stock_moving_avg-20           2874.6238   5052.592      0.569      0.569   -7037.728    1.28e+04
LME_Zinc_moving_avg-3                4492.9682   8533.078      0.527      0.599   -1.22e+04    2.12e+04
LME_Zinc_moving_avg-5               -3987.3050   1.87e+04     -0.21

In [482]:
temp_df

,Freight Rate,LME_Aluminium_moving_avg-3,LME_Aluminium_moving_avg-5,LME_Aluminium_moving_avg-7,LME_Aluminium_moving_avg-10,LME_Aluminium_moving_avg-15,LME_Aluminium_moving_avg-20,LME_Al_Stock_moving_avg-3,LME_Al_Stock_moving_avg-5,LME_Al_Stock_moving_avg-7,...,Crude_oil_Brent_price_moving_avg-7,Crude_oil_Brent_price_moving_avg-10,Crude_oil_Brent_price_moving_avg-15,Crude_oil_Brent_price_moving_avg-20,Crude_oil_Brent_Vol_moving_avg-3,Crude_oil_Brent_Vol_moving_avg-5,Crude_oil_Brent_Vol_moving_avg-7,Crude_oil_Brent_Vol_moving_avg-10,Crude_oil_Brent_Vol_moving_avg-15,Crude_oil_Brent_Vol_moving_avg-20
20,1590.000000,0.312496,0.328444,0.343355,0.346801,0.349504,0.352407,0.622900,0.623297,0.623372,...,0.450971,0.439247,0.425974,0.433346,0.207555,0.254631,0.238961,0.280108,0.497856,0.603398
21,1600.000000,0.305404,0.325702,0.336486,0.347744,0.348774,0.350830,0.621637,0.624484,0.625857,...,0.451126,0.440681,0.426162,0.431587,0.113644,0.169667,0.205380,0.188589,0.368545,0.605089
22,1593.333333,0.299139,0.321262,0.330351,0.346778,0.347979,0.349038,0.620715,0.625428,0.627022,...,0.449117,0.440176,0.425481,0.428455,0.195887,0.203877,0.223087,0.202261,0.367265,0.606073
23,1586.666667,0.295215,0.315517,0.327412,0.344686,0.348043,0.347030,0.621637,0.624496,0.627754,...,0.447631,0.439701,0.426579,0.426190,0.282100,0.229023,0.230097,0.209885,0.335409,0.596174
24,1580.000000,0.291979,0.310860,0.323642,0.338661,0.348218,0.345429,0.621262,0.623990,0.628468,...,0.446429,0.439570,0.428629,0.425071,0.407977,0.257318,0.232853,0.248412,0.314204,0.578059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,1054.166667,0.299754,0.307647,0.319111,0.334733,0.333638,0.328154,0.153329,0.152792,0.151415,...,0.574713,0.575841,0.568828,0.561392,0.457565,0.451395,0.512008,0.566546,0.581226,0.526739
1382,1058.333333,0.301252,0.311143,0.316430,0.334987,0.335546,0.328430,0.151608,0.152236,0.151556,...,0.575974,0.575116,0.570733,0.564277,0.471105,0.494873,0.520938,0.567956,0.598825,0.593739
1383,1062.500000,0.301769,0.315181,0.316535,0.332004,0.337695,0.329998,0.149820,0.151872,0.152397,...,0.575849,0.573279,0.572833,0.567076,0.494116,0.519674,0.514977,0.548870,0.652555,0.660819
1384,1066.666667,0.300013,0.316400,0.319301,0.328839,0.339924,0.331424,0.149967,0.151620,0.152563,...,0.577726,0.572215,0.573074,0.569183,0.479481,0.497383,0.525008,0.541360,0.641380,0.726434


In [483]:
split_point = len(temp_df) - 10
print(split_point)

X_train = X_multi.iloc[:split_point]
X_test = X_multi.iloc[split_point:]

y_train = y_multi.iloc[:split_point]
y_test = y_multi.iloc[split_point:]
    
print ('Shape of the datasets \nTraining Dataset (X): {}\nTesting Dataset (X): {}\nTraining Dataset (y): {}\nTesting Dataset (y): {}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

1356
Shape of the datasets 
Training Dataset (X): (1356, 96)
Testing Dataset (X): (10, 96)
Training Dataset (y): (1356,)
Testing Dataset (y): (10,)


In [484]:
# Fitting Random Forest Regression to the dataset
# import the regressor

from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

# fit the regressor with x and y data
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [485]:
for dt, pred in zip(last_10_days[10:]['Date'], regressor.predict(X_test)):
    print (dt, pred)

2023-07-19 1138.6902777777768
2023-07-20 1135.928174603174
2023-07-21 1145.0652777777775
2023-07-22 1157.2279761904758
2023-07-23 1162.0107142857141
2023-07-24 1168.6523809523808
2023-07-25 1167.265476190476
2023-07-26 1168.2369047619045
2023-07-27 1167.805753968254
2023-07-28 1166.9251984126981


In [486]:
temp_df_del = temp_df.copy()

In [487]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

features = list(temp_df_del.columns)[1:]
target = 'Freight Rate'

data = temp_df_del.copy()

train = data[:split_point]
test = data[split_point:]

print (train.shape)
print (test.shape)

# Split data into training, validation, and testing sets
train, val = train_test_split(train, test_size=0.2, random_state=42)

# Create XGBoost Regressor model
model = xgb.XGBRegressor()

# Train model on training set
model.fit(train[features], train[target],
          eval_set=[(val[features], val[target])],
          early_stopping_rounds=10, verbose=False)

# Make predictions on validation set
val_preds = model.predict(val[features])

# Calculate mean squared error on testing set
mse = mean_squared_error(val[target], val_preds)
print('Mean Squared Error for Validation Set:', mse)

# Make predictions on testing set
test_preds = model.predict(test[features])

# Calculate mean squared error on testing set
mse = mean_squared_error(test[target], test_preds)
# print('Mean Squared Error:', mse)

(1356, 97)
(10, 97)


C:\Users\218934\Anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Mean Squared Error for Validation Set: 46198.387014295455


In [488]:
for dt, pred in zip(last_10_days[10:]['Date'], test_preds):
    print (dt, pred)

2023-07-19 1123.8601
2023-07-20 1123.8601
2023-07-21 1123.8601
2023-07-22 1123.8601
2023-07-23 1188.4537
2023-07-24 1191.6697
2023-07-25 1200.9326
2023-07-26 1188.4537
2023-07-27 1188.4537
2023-07-28 1191.8595


In [489]:
test[features]

,LME_Aluminium_moving_avg-3,LME_Aluminium_moving_avg-5,LME_Aluminium_moving_avg-7,LME_Aluminium_moving_avg-10,LME_Aluminium_moving_avg-15,LME_Aluminium_moving_avg-20,LME_Al_Stock_moving_avg-3,LME_Al_Stock_moving_avg-5,LME_Al_Stock_moving_avg-7,LME_Al_Stock_moving_avg-10,...,Crude_oil_Brent_price_moving_avg-7,Crude_oil_Brent_price_moving_avg-10,Crude_oil_Brent_price_moving_avg-15,Crude_oil_Brent_price_moving_avg-20,Crude_oil_Brent_Vol_moving_avg-3,Crude_oil_Brent_Vol_moving_avg-5,Crude_oil_Brent_Vol_moving_avg-7,Crude_oil_Brent_Vol_moving_avg-10,Crude_oil_Brent_Vol_moving_avg-15,Crude_oil_Brent_Vol_moving_avg-20
1376,0.315043,0.330011,0.326613,0.325139,0.328457,0.328904,0.147586,0.149658,0.151524,0.153687,...,0.581701,0.571033,0.555402,0.549587,0.489024,0.566548,0.584037,0.529795,0.381118,0.416398
1377,0.299605,0.326997,0.327692,0.326053,0.327547,0.327850,0.147319,0.149541,0.150929,0.152928,...,0.581418,0.571881,0.557806,0.549786,0.458994,0.524997,0.572786,0.518912,0.420371,0.420997
1378,0.289931,0.317312,0.327420,0.327031,0.328159,0.326279,0.149191,0.149326,0.150807,0.152470,...,0.579845,0.573667,0.560550,0.552018,0.428877,0.474105,0.556178,0.564643,0.471195,0.410243
1379,0.286168,0.310102,0.327079,0.329614,0.329610,0.326510,0.151908,0.149647,0.151069,0.152234,...,0.579592,0.575142,0.564046,0.555290,0.415777,0.486130,0.549280,0.567664,0.549554,0.452170
1380,0.293672,0.307697,0.322695,0.332897,0.331528,0.327738,0.153719,0.151493,0.151429,0.152368,...,0.575930,0.576235,0.566910,0.557760,0.443157,0.472578,0.511745,0.572760,0.544364,0.490097
1381,0.299754,0.307647,0.319111,0.334733,0.333638,0.328154,0.153329,0.152792,0.151415,0.152162,...,0.574713,0.575841,0.568828,0.561392,0.457565,0.451395,0.512008,0.566546,0.581226,0.526739
1382,0.301252,0.311143,0.316430,0.334987,0.335546,0.328430,0.151608,0.152236,0.151556,0.151821,...,0.575974,0.575116,0.570733,0.564277,0.471105,0.494873,0.520938,0.567956,0.598825,0.593739
1383,0.301769,0.315181,0.316535,0.332004,0.337695,0.329998,0.149820,0.151872,0.152397,0.151528,...,0.575849,0.573279,0.572833,0.567076,0.494116,0.519674,0.514977,0.548870,0.652555,0.660819
1384,0.300013,0.316400,0.319301,0.328839,0.339924,0.331424,0.149967,0.151620,0.152563,0.151563,...,0.577726,0.572215,0.573074,0.569183,0.479481,0.497383,0.525008,0.541360,0.641380,0.726434
1385,0.301908,0.316350,0.321988,0.327542,0.342173,0.332853,0.150912,0.150542,0.152302,0.151952,...,0.579119,0.572996,0.574174,0.571670,0.466920,0.512027,0.545078,0.542159,0.654677,0.730312
